# State tomography

Techniques for perfoming a quantum state tomography in Pyquil. State preparation is assumed to be implemented by a specified Pyquil program.

WRONG: Given a Pyquil program, it assumes that the program implements a quantum operation, and runs a state tomography protocol. It applies the specified program to different quantum states, measure it in different basis and reconstruct the quantum density matrix.

In [1]:
import pylab as py
import qutip as qp
from pyquil.quil import Program
import pyquil.api as api
import pyquil.gates as gt
qvm = api.QVMConnection()
import math,cmath,itertools,scipy.linalg

# __all__ = [
#     'main',
#     ]

In [2]:
def main(program, indices):
    """
    State tomography of the state prepared by 'program'
    on qubits indexed by 'indices'
    """
    return Tomography(indices).perform(program)

In [3]:
class Tomography():
    """
    Functions necessary to perform state tomography
    """

    def __init__(self, n_qubits, qubit_order='rigetti'):
        assert isinstance(n_qubits,int), u"Variable 'n_qubits' must be an integer"
        assert n_qubits > 0, u"Variable 'n_qubits' must be positive"
        assert qubit_order.lower() in ['rigetti','qinfo'], u"Variable 'qubit_order' must be 'rigetti' or 'qinfo'"
        self.n_qubits = n_qubits
        self.qubit_order = qubit_order
        self.qutip_basis = [qutip_representation(x,self.qubit_order) for x in self.gen_basis()]
        
    def gen_basis(self):
        """
        Generate basis for the space of density matrices
        """
        if self.qubit_order == 'rigetti':
            order = -1
        elif self.qubit_order == 'qinfo':
            order = +1
        for index in itertools.product('izxy',repeat=self.n_qubits):
            yield ''.join(index[::order])

    def gen_basis_choice(self):
        """
        Generate a complete set of measurement basis
        """
        if self.qubit_order == 'rigetti':
            order = -1
        elif self.qubit_order == 'qinfo':
            order = +1
        for index in itertools.product('zxy',repeat=self.n_qubits):
            yield ''.join(index[::order])
    
    def gen_observables(self,basis_choice):
        """
        Generate observables for a quantum measurement on a specific local basis
        """
        for index in itertools.product(range(2),repeat=self.n_qubits):
            yield ''.join(['i' if flag==0 else basis for basis,flag in zip(basis_choice,index)][::+1])

    def build_design_matrix(self):
        """
        Build a design matrix for least-square optimization
        """
        lines = []
        hadamard = scipy.linalg.hadamard(2**self.n_qubits)
        for basis_choice in self.gen_basis_choice():
            block = []
            for measure in self.gen_observables(basis_choice):
                this = [calc_dotproduct(measure,b) for b in self.gen_basis()]
                block.append(this)
            block = py.dot(hadamard,py.array(block)) / 2**self.n_qubits
            for line in block:
                lines.append(list(line))
        return py.array(lines)

    def calculate(self, result):
        """
        Calculate the quantum state based on measured results
        """
        self.latest_result = py.dot(self.design_matrix_pinv, py.array(result))
        return self.latest_result  

    def get_qutip(self, result=None):
        """
        Get a qutip representation of the quantum density matrix.
        If 'result' is speficied, it calculates the tomography.
        If 'result' is not specified, it uses the last known result
        """
        if result is not None:
            array = self.calculate(result)
        else:
            array = self.latest_result
        rho = sum([x*y for x,y in zip(array,self.qutip_basis)])
        return rho

In [4]:
class TomographyForest(Tomography):
    """
    Functions necessary to perform state tomography on a Pyquil program
    using run command and data analysis
    """

    def __init__(self, qvm, indices, n_runs = None):
        self.rigetti_machine = qvm
        indices = list(indices)
        assert isinstance(list(indices),list), u"Variable 'indices' must be convertable to a list"
        for number in indices:
            assert isinstance(number,int), u"Members of 'indices' must be integers"
        Tomography.__init__(self, len(indices))
        self.qubits = indices
        self.n_runs = n_runs or 1000 * 2**(self.n_qubits)
        self.circuits_for_basis_change = [x for x in self.gen_circuits_for_basis_change()]
        self.design_matrix = self.build_design_matrix()
        self.design_matrix_pinv = py.pinv(self.design_matrix)

    
    def change_of_basis(self, index, theta, phi):
        """
        Create a Pyquil list of gates to convert a Z-measurement
        into a (theta,phi)-measurement
        """
        gates = []
        gates.append(gt.RZ(-phi, index))
        gates.append(gt.RX(+py.pi/2, index))
        gates.append(gt.RZ(-theta, index))
        gates.append(gt.RX(-py.pi/2, index))
        return gates
        
    def gen_circuits_for_basis_change(self):
        change = {}
        change['z'] = lambda index: self.change_of_basis(index, 0.0, 0.0)
        change['x'] = lambda index: self.change_of_basis(index, +py.pi/2, 0.0)
        change['y'] = lambda index: self.change_of_basis(index, +py.pi/2, +py.pi/2)
        for basis_choice in self.gen_basis_choice():
            program = Program()
            for basis,index in zip(basis_choice, self.qubits):
                gates = change[basis](index)
                for gate in gates:
                    program.inst(gate)
            yield program
            
    def aggregate_results(self, results, as_dict = False):
        aggregate = {}
        for result in results:
            this = int(''.join([str(x) for x in result]),base = 2)
            if this not in aggregate:
                aggregate[this] = 1
            else:
                aggregate[this] += 1
        if as_dict:
            return aggregate
        else:
            return [aggregate.get(x,0) for x in range(2**self.n_qubits)]
            
    def perform(self, program):
        results = []
        for change in self.circuits_for_basis_change:
            measure = Program()
            for k,index in enumerate(self.qubits):
                # Add measurements
                measure.measure(index, k)
            to_run = program + change + measure
            classical_addresses = range(len(self.qubits))
            trials = self.n_runs
            result = self.rigetti_machine.run(to_run, classical_addresses, trials)
            aggregate = self.aggregate_results(result)
            results += list(aggregate)
        rho = self.get_qutip(results) / self.n_runs
        bloch = self.latest_result / self.latest_result[0]
        return rho, bloch

In [5]:
class TomographyWavefunction(Tomography):
    """
    Functions necessary to perform state tomography on a Pyquil program
    using wavefunction command
    """

    def __init__(self, qvm, n_qubits):
        Tomography.__init__(self, n_qubits)
        self.rigetti_machine = qvm

    def perform(self, program):
        wavefunction = self.rigetti_machine.wavefunction(program)
        amplitudes = list(wavefunction.amplitudes)
        amplitudes += [0] * (2**self.n_qubits - len(amplitudes))
        ket = qp.Qobj(py.array(amplitudes))
        dims = [[2]*self.n_qubits]*2
        rho = qp.Qobj(qp.ket2dm(ket),dims = dims)
        bloch = [qp.expect(basis,rho) for basis in self.qutip_basis]
        return rho, bloch

In [6]:
def calc_dotproduct(pauliA,pauliB):
    """
    Calculate the trace of the product of Pauli operators
    Input:
        pauliA, pauliB : strings with chars in the set 'IXYZ'
    Output:
        a real number
    """
    assert len(pauliA) == len(pauliB), u"Variables 'pauliA' and 'pauliB' must have the same lenght"
    assert all([x.lower() in 'ixyz' for x in pauliA]), u"Members of 'pauliA' must be in the set 'ixyz'"
    assert all([x.lower() in 'ixyz' for x in pauliB]), u"Members of 'pauliB' must be in the set 'ixyz'"
    if any([x.lower()!=y.lower() for x,y in zip(pauliA,pauliB)]):
        return 0
    else:
        return 2**len(pauliA)

def qutip_representation(pauliA, qubit_order):
    """
    Get a qutip representation of an operator
    Input:
        pauliA : strings with chars in the set 'IXYZ'
    Output:
        a qutip object
    """
    assert all([x.lower() in 'ixyz' for x in pauliA]), u"Members of pauliA must be in the set 'ixyz'"
    assert qubit_order.lower() in ['rigetti','qinfo'], u"Variable 'qubit_order' must be 'rigetti' or 'qinfo'"
    terms = [pauli_matrices[x.lower()] for x in pauliA]
    if qubit_order == 'rigetti':
        rho = qp.tensor(terms[::-1])
    elif qubit_order == 'qinfo':
        rho = qp.tensor(terms[::+1])
    return rho

pauli_matrices = {}
pauli_matrices['i'] = qp.qeye(2)
pauli_matrices['x'] = qp.sigmax()
pauli_matrices['y'] = qp.sigmay()
pauli_matrices['z'] = qp.sigmaz()


This concludes the state tomography.

# Testing area

In [7]:
testing = False

In [8]:
if testing:
    t = TomographyWavefunction(qvm,2)
    rho_A,bloch_A = t.perform(Program([gt.H(1),gt.H(0),gt.S(1)]))
    t = TomographyForest(qvm,[0,1])
    rho_B,bloch_B = t.perform(Program([gt.H(1),gt.H(0),gt.S(1)]))
    print(rho_A)
    print(rho_B)
    print((rho_A - rho_B).norm())

In [9]:
if testing:

    for x in bloch_A: print(x)
    print('\n\n')
    for x in bloch_B: print(x)

        

In [10]:
if testing:

    import random
    gates = []
    gates.append(gt.RX(2*py.pi*random.random(),0))
    gates.append(gt.RZ(2*py.pi*random.random(),0))
    gates.append(gt.RX(2*py.pi*random.random(),1))
    gates.append(gt.RZ(2*py.pi*random.random(),1))
    gates.append(gt.RX(2*py.pi*random.random(),2))
    gates.append(gt.RZ(2*py.pi*random.random(),2))
    program = Program(gates)
    tomo_A = TomographyWavefunction(qvm, 3)
    tomo_B = TomographyForest(qvm, [0,1,2])
    rho_A,bloch_A = tomo_A.perform(program)
    rho_B,bloch_B = tomo_B.perform(program)



In [11]:
if testing:
    print( py.norm((rho_A - rho_B).full()) )
    print( py.norm(py.array(bloch_A)-py.array(bloch_B)) )



In [12]:
if testing:


    max(abs(py.array(bloch_A)-py.array(bloch_B)))

In [13]:
if testing:


    bloch_A

In [14]:
if testing:
    program = Program([gt.X(0),gt.X(0),gt.X(3),gt.X(3),gt.X(2),gt.X(2),gt.X(0)])
    wv = qvm.wavefunction(program)
    print(wv.amplitudes)
    print(wv.pretty_print())

In [15]:
if testing:

    tomo = Tomography(2)

In [16]:
if testing:

    [x for x in tomo.gen_basis()]

In [17]:
if testing:

    [x for x in tomo.gen_basis_choice()]

In [18]:
if testing:

    [x for x in tomo.gen_observables('zz')]

# Process tomography

In [128]:
class ProcessTomography(Tomography):

    def gen_initial_states(self):
        """
        Generate initial states for process tomography.
        """
        for direction in self.gen_basis_choice():
            for signal in itertools.product('+-',repeat = self.n_qubits):
                yield [''.join([x,y]) for x,y in zip(direction,signal)]
      
                
                
    def build_design_matrix(self):
        lines = []
        for vector_k,vector_j in itertools.product(self.gen_basis(),repeat=2):
            line = []
            for vector_state in self.gen_initial_states():
                for vector_s in self.gen_basis():
                    for k,j,s,st in zip(vector_k,vector_j,vector_s,vector_state):
                        element = calculate_tripletrace(k,j,s,st)
                        line.append(element)
                        print(element)
                        
            lines.append(line)
        self.design_matrix = py.array(lines).T
        
    
def calculate_tripletrace_qutip(element_k,element_j,element_s,element_state):
    sigma = pauli_matrices[element_k]
    sigma = pauli_matrices[element_s] * sigma
    sigma = pauli_matrices[element_j] * sigma
    state = pauli_states[element_state]
    expect = qp.expect(sigma,state)
    return expect

def calculate_tripletrace(element_k,element_j,element_s,element_state):
    non_identity = [x.lower() for x in [element_j,element_s,element_k] if x.lower()!='i']
    if len(non_identity) == 0:
        return +1
    elif len(non_identity) == 1:
        if element_state[0] == non_identity[0]:
            return +1 if element_state[0] == '+' else -1
        else:
            # IMPROVE
            sigma = pauli_matrices[non_identity[0]]
            state = pauli_states[element_state]
            return qp.expect(sigma,state)
    elif len(non_identity) == 2:
        if non_identity[0] == non_identity[1]:
            return +1
        else:
            # IMPROVE
            sigma = pauli_matrices[non_identity[0]] * pauli_matrices[non_identity[1]]
            state = pauli_states[element_state]
            return qp.expect(sigma,state)
    elif len(non_identity) == 3:
        # IMPROVE
        sigma = pauli_matrices[non_identity[0]] * pauli_matrices[non_identity[1]]
        state = pauli_states[element_state]
        return qp.expect(sigma,state)



        
            



pauli_states = {}
pauli_states['z+'] = qp.basis(2,0)
pauli_states['z-'] = qp.basis(2,1)
pauli_states['x+'] = (qp.basis(2,0) + 1 * qp.basis(2,1)).unit()
pauli_states['x-'] = (qp.basis(2,0) - 1 * qp.basis(2,1)).unit()
pauli_states['y+'] = (qp.basis(2,0) + 1j * qp.basis(2,1)).unit()
pauli_states['y-'] = (qp.basis(2,0) - 1j * qp.basis(2,1)).unit()




